# Build a simple RAG system with semantic search and BM25

To build a simple RAG, we need build a query function, a retrieval function, and a generation function. We will also need a dataset to retrieve from.




## Understand Data Structures

Many tutorials usually use some sort of open-source dataset as examples to show how RAG works. For me, the problem is not really about the RAG workflow, but how to work with data. For beginners like me, it is not a simple question. It is actually the most mind-boggling part of the whole process.



In [ ]:
!pip install bm25s
!pip install nltk
!pip install scikit-learn
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np
import bm25s
import os
from sentence_transformers import SentenceTransformer

First we need to read the dataset. Pandas is a great and most common choice for data manipulation. We can use `pd.read_csv()` to read the dataset into a DataFrame and assign it to a variable. 

Then we can do some basic data exploration to understand the structure of the dataset. For example, we can use `df.head()` to see the first few rows of the dataset, and `df.info()` to see the data types and null values in each column. Here "df" is the DataFrame variable we assigned earlier, it can be any name you choose but somehow people like to use "df" as a short form for DataFrame.





In [ ]:
NEWS_DATA = pd.read_csv("./resource/sample_data/news_data_dedup.csv")
NEWS_DATA.head(3) # Display the first few rows of the dataset

In [ ]:
NEWS_DATA.info()

In [ ]:
NEWS_DATA['description'][0] #access the first row of the description column

In [ ]:
NEWS_DATA.loc[0] #access the first row

It takes some practice to become comfortable with accessing and manipulating data in Pandas. Don't panic.

Pandas library has very good documentation here: https://pandas.pydata.org/

And a cheat sheet here: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf . A copy of the cheat sheet is also included in the resource folder of this project. 


## Start with querying

If we have a structured dataset, we can start with querying by index. We can create a function that takes an index or a list of indices as input and returns the corresponding rows from the DataFrame. This is a simple way to retrieve data without any complex logic.

In [ ]:
def query_news(indices, dataset=NEWS_DATA):
    """
    Retrieves elements from a dataset based on specified indices.

    Parameters:
    indices (list of int): A list containing the indices of the desired elements in the dataset.
    dataset (list or sequence): The dataset from which elements are to be retrieved. It should support indexing.

    Returns:
    list: A list of elements from the dataset corresponding to the indices provided in list_of_indices.
    """

    output = [dataset.iloc[index] for index in indices]

    return output

# Example usage
indices = [0, 1, 2]
result = query_news(indices)
print(result)
print("Type of result:", type(result))
print("=" * 50)
# Display the first element's description from the result
print(f"""Description of {result[0]['title']}:\n 
\"{result[0]['description']} \"\n 
Publication date: {result[0]['published_at']}""")  



### BM25 Retrieval

Now that we can retrieve data by index, we can work on the retrieval function, which ideally should take a query as input and return the indices of the relevant rows from the DataFrame. 

Let's start with bm25 using `bm25s` library. BM25 is a popular algorithm for information retrieval that ranks documents based on their relevance to a given query. It is widely used in search engines and can be a good starting point for building a retrieval function.

First, we will construct the "corpus" from the DataFrame. The corpus is a list of strings, where each string is a document in the dataset.

In [ ]:
# The corpus used will be the title appended with the description
# the "to_dict('records')" method converts the DataFrame to a list of dictionaries, where each dictionary represents a row in the DataFrame.
records = NEWS_DATA.to_dict('records')
print(records[0])  # Display the first record to check the structure
corpus = [x['title'] + " " + x['description'] for x in records]
print("Corpus created with", len(corpus), "documents. The type of corpus is:", type(corpus))
print("First document in the corpus:", corpus[0])

We then need to instantiate the BM25 retriever by passing the corpus data. The `bm25s` library provides a `BM25` class that we can use for this purpose.

In the follow code, the `BM25_RETRIEVER` is an instance of the `BM25` class, which is initialized with the corpus. There are some common methods we can use with the `BM25_RETRIEVER` object, such as `index()` to index the tokenized chunks, and `retrieve()` to retrieve relevant documents based on a query.

To instantiate the retriever, we first need call the `bm25s.BM25()` constructor with the corpus data. 

Then we can tokenize the corpus data using the `bm25s.tokenize()` function. This function takes the corpus and an optional list of stopwords as input and returns a list of tokenized documents. Tokenization is the process of breaking down the text into smaller units (tokens) for better processing.

The next step is to index the tokenized chunks within the retriever. This will allow the retriever to efficiently search for relevant documents based on a query. The `index()` method is used for this purpose, and it takes the tokenized data as input.

After tokenizing and indexing the data, we are ready to use the BM25 retriever to retrieve relevant documents based on a query. The `retrieve()` method can be used to perform the retrieval, and it will return a list of relevant documents along with their scores.

In [ ]:
# Instantiate the retriever by passing the corpus data
# BM25_RETRIEVER is an instance of the BM25 class from the bm25s library, which is used for information retrieval.
# It is initialized with the corpus, which is a list of strings where each string represents a document in the dataset.
BM25_RETRIEVER = bm25s.BM25(corpus=corpus)

# Define a list of stopwords to be used during tokenization, this is optional but may improve retrieval performance
stopwords = ["a", "the", "and", "is", "to", "of", "in", "that", "it", "for", "on", "with", "as", "this", "by", "at", "from"]
# Tokenize the chunks, which means breaking down the text into smaller units (tokens) for better processing
tokenized_data = bm25s.tokenize(corpus, stopwords=stopwords)

# Index the tokenized chunks within the retriever, this is done automatically when you call the `BM25` constructor, but you can also do it explicitly
BM25_RETRIEVER.index(tokenized_data)

# Check the content of tokenized_data
print("Content of tokenized_data:", tokenized_data[:3])

In [ ]:
# Tokenize the same query used in the previous exercise
sample_query = "What are the recent news about GDP?"
tokenized_sample_query = bm25s.tokenize(sample_query)

# Get the retrieved results and their respective scores
results, scores = BM25_RETRIEVER.retrieve(tokenized_sample_query, k=5)

"""
Note the actual results and scores are 'lists of lists', where each inner list corresponds to a query. 
Since we only have one query, we can access the first element of each list by calling results[0] and scores[0].
"""

for r,s in zip(results[0], scores[0]):
    print(f"Document: {r} \n Score: {s}")

print(f"Results for query: {sample_query}\n")
for doc in results[0]:
  print(f"Document retrieved {corpus.index(doc)} : {doc}\n")

In [ ]:
# we need the doc numbers(indices) to perform fusion ranking later
indices_bm25 = [corpus.index(doc) for doc in results[0]]
print("Indices of retrieved documents:", indices_bm25)

!NOTE 

The output from the BM25 retriever will be the same for the same query, regardless of the order of the documents in the corpus and the word order of the query. This is because BM25 is a statistical model that calculates the relevance of each document to the query based on the term frequency and inverse document frequency. In comparison, a semantic search model might produce different results based on the context and meaning of the words in the query and documents. Order of the documents in the corpus and the word order matters for semantic search, but not for BM25.





### Semantic Search Retrieval
Now that we have a basic understanding of how to retrieve data by index and using BM25, we can move on to semantic search. Semantic search is a more advanced technique that uses natural language processing (NLP) to understand the meaning of the query and the documents in the corpus. It can provide more relevant results than BM25, especially for complex queries.

A key component of semantic search is the use of embeddings, which are vector representations of text. These embeddings capture semantic meaning, allowing us to compare text based on context. One common way to measure the similarity between these vectors is through cosine similarity, which calculates how close two vectors are in high-dimensional space. This approach helps in finding content that is contextually similar to the user's query, leading to more accurate and meaningful search results.

In contrast, BM25 uses a sparse representation based on keyword matching. During the indexing stage, BM25 tokenizes documents, builds an inverted index (mapping each term to the documents it appears in along with term frequencies), and stores statistics such as document length. During the retrieval stage, BM25 tokenizes the query in the same way, looks up documents containing those terms in the inverted index, and calculates a relevance score using the BM25 formula, which combines term frequency (TF), inverse document frequency (IDF), and length normalization.

In [ ]:
!pip install joblib

In [ ]:
import joblib
from sentence_transformers import SentenceTransformer

In [ ]:
# for the first time, we need to create embeddings for the corpus
# for the purpose of this example, we will load pre-computed embeddings
EMBEDDINGS = joblib.load("./resource/sample_data/embeddings.joblib")

`joblib` is a library that provides tools for saving and loading Python objects, including NumPy arrays and scikit-learn models. It is particularly useful for efficiently storing large datasets or machine learning models in a binary format, allowing for quick loading and saving without the need to re-compute or re-train.

You can find the `joblib` library documentation here: https://joblib.readthedocs.io/en/latest/

In [ ]:
model_name = "BAAI/bge-base-en-v1.5" 
model = SentenceTransformer(model_name)

In [ ]:
# Example usage
query = "RAG is awesome"
# Using, but truncating the result to not pollute the output, don't truncate it in the exercise.
model.encode(query)[:40]

In [ ]:
def cosine_similarity(a, b):
    """
    Calculate the cosine similarity between two vectors.

    Parameters:
    a (np.ndarray): First vector.
    b (np.ndarray): Second vector.

    Returns:
    float: Cosine similarity between the two vectors.
    """
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))   
    

In [ ]:
query1 = "What are the primary colors"
query2 = "Yellow, red and blue"
query3 = "Cats are friendly animals"

query1_embed = model.encode(query1)
query2_embed = model.encode(query2)
query3_embed = model.encode(query3)

print(f"Similarity between '{query1}' and '{query2}' = {cosine_similarity(query1_embed, query2_embed)}")
print(f"Similarity between '{query1}' and '{query3}' = {cosine_similarity(query1_embed, query3_embed)}")

In [ ]:
def semantic_search(query, embeddings, model, top_k=5):
    """
    Perform semantic search to find the most relevant documents to a given query.

    Parameters:
    query (str): The search query.
    embeddings (np.ndarray): A 2D array where each row is the embedding of a document in the corpus.
    model (SentenceTransformer): A pre-trained SentenceTransformer model used to encode the query.
    top_k (int): The number of top relevant documents to return.

    Returns:
    list: Indices of the top_k most relevant documents in the corpus.
    """
    
    # Encode the query to get its embedding
    query_embedding = model.encode(query)
    
    # Compute cosine similarities between the query embedding and all document embeddings
    # the input embeddings is a 2D array, where each row is a document embedding
    # the query_embedding is a 1D array
    # we use broadcasting to compute the cosine similarity between the query and all documents
    cosine_similarities = np.dot(embeddings, query_embedding) / (np.linalg.norm(embeddings, axis=1) * np.linalg.norm(query_embedding))
    
    # Get the indices of the top_k most similar documents
    top_k_indices = np.argsort(cosine_similarities)[-top_k:][::-1]
    
    return top_k_indices.tolist()   

The broadcasting mechanism in NumPy allows us to perform operations on arrays of different shapes. In this case, we can compute the cosine similarity between the query embedding and all document embeddings without needing to reshape the arrays explicitly. A step-by-step breakdown of the cosine similarity calculation is as follows:

1. `dot_product = np.dot(EMBEDDINGS, query_embed)`: This computes the dot product between the query embedding and each document embedding. For this particular model, say if we have 1000 documents and each embedding is a 768-dimensional vector, the `EMBEDDINGS` array would have a shape of (1000, 768), and the `query_embed` would have a shape of (768,). The resulting `dot_product` would be a 1D array of shape (1000,), where each element represents the dot product between the query and a corresponding document.

2. `norms = np.linalg.norm(EMBEDDINGS, axis=1) * np.linalg.norm(query_embed)`: This computes the norms of the document embeddings and the query embedding. For 1000 documents, the `norms` array would have a shape of (1000,), where each element represents the norm of a corresponding document embedding multiplied by the norm of the query embedding. The query embedding's norm is a single scalar value and it is the same for all documents, so it can be broadcasted across the document norms.

3. `cosine_similarities = dot_product / norms`: This computes the cosine similarities by dividing the dot products by the norms. The result is a 1D array where each element represents the cosine similarity between the query and a corresponding document.

demostration of dot matrix multiplication and cosine similarity calculation with  3*2 matrix and 2*1 vector:


```math
\begin{align*}
\text{Dot Product} & : \begin{bmatrix}
1 & 2 \\
3 & 4 \\
5 & 6
\end{bmatrix} \cdot \begin{bmatrix}
7 \\
8
\end{bmatrix} = \begin{bmatrix}
1 \cdot 7 + 2 \cdot 8 \\
3 \cdot 7 + 4 \cdot 8 \\
5 \cdot 7 + 6 \cdot 8
\end{bmatrix} = \begin{bmatrix}
23 \\
83 \\
143
\end{bmatrix} \\

\end{align*}

In [ ]:
query = "trend of economy"
top_k_indices_semantic = semantic_search(query, EMBEDDINGS, model, top_k=7)
print(f"Top {len(top_k_indices_semantic)} indices for query '{query}': {top_k_indices_semantic}")

In [ ]:
#run bm25 again
results, scores = BM25_RETRIEVER.retrieve(bm25s.tokenize(query), k=7)
indices_bm25 = [corpus.index(doc) for doc in results[0]]
print("Indices of retrieved documents:", indices_bm25)

## RRF

RRF (Reciprocal Rank Fusion) is a method that combines the results from multiple retrieval models to improve the overall performance of the retrieval system. It works by assigning a score to each document based on its rank in the results of different models, and then combining these scores to produce a final ranking.

The RRF algorithm works as follows:
1. For each retrieval model, retrieve a ranked list of documents for a given query.

2. For each document in the ranked list, assign a score based on its rank. The score is typically calculated as `1 / (k + rank)`, where `k` is a constant (often set to 60) and `rank` is the position of the document in the ranked list.

3. Sum the scores for each document across all retrieval models.

4. Rank the documents based on their combined scores to produce the final result.

The formula for the RRF score of a document `d` is given by:

```math
\text{RRF}(d) = \sum_{i=1}^{N} \frac{1}{k + \text{rank}_i(d)}
```



In [ ]:
def reciprocal_rank_fusion(list1, list2, top_k=5, K=50):
    """
    Fuse rank from multiple IR systems using Reciprocal Rank Fusion.

    Args:
        list1 (list[int]): A list of indices of the top-k documents that match the query.
        list2 (list[int]): Another list of indices of the top-k documents that match the query.
        top_k (int): The number of top documents to consider from each list for fusion. Defaults to 5.
        K (int): A constant used in the RRF formula. Defaults to 60.

    Returns:
        list[int]: A list of indices of the top-k documents sorted by their RRF scores.
    """

    ### START CODE HERE ###

    # Create a dictionary to store the RRF scores for each document index
    rrf_scores = {}

    # Iterate over each document list
    for lst in [list1, list2]:
        # Calculate the RRF score for each document index
        for rank, item in enumerate(lst, start=1): # Start = 1 set the first element as 1 and not 0. 
                                                   # This is a convention on how ranks work (the first element in ranking is denoted by 1 and not 0 as in lists)
            # If the item is not in the dictionary, initialize its score to 0
            if item not in rrf_scores:
                rrf_scores[item] = 0
                
            # Update the RRF score for each document index using the formula 1 / (rank + K)
            current_score = 1/(rank+K)
            
            #print(f"Document {item} from {lst} has RRF score: {current_score:.4f}")
            rrf_scores[item] += current_score
            #print(f"Updated RRF score for document {item}: {rrf_scores[item]:.4f}")

    # Sort the document indices based on their RRF scores in descending order
    sorted_items = sorted(rrf_scores, key=rrf_scores.get, reverse = True)

    # Slice the list to get the top-k document indices
    top_k_indices = [int(x) for x in sorted_items[:top_k]]

    ### END CODE HERE ###

    return top_k_indices


In [ ]:

# Example usage of reciprocal_rank_fusion
top_k_indices_bm25 = indices_bm25[:5]  # Assuming we want the top 5 from BM25
top_k_indices_semantic = top_k_indices_semantic[:5]  # Assuming we want the top 5 from semantic search
top_k_indices_fused = reciprocal_rank_fusion(top_k_indices_bm25, top_k_indices_semantic, top_k=5)

print(f"Top {len(top_k_indices_bm25)} indices from BM25: {top_k_indices_bm25}")
print(f"Top {len(top_k_indices_semantic)} indices from semantic search: {top_k_indices_semantic}")
print(f"Top {len(top_k_indices_fused)} indices after RRF fusion: {top_k_indices_fused}")    

In [ ]:
# retrieve the documents using the fused indices
fused_documents = query_news(top_k_indices_fused, NEWS_DATA)
print("Fused documents retrieved:")
for doc in fused_documents:

    print(f"Title: {doc['title']},\n Description: {doc['description']}, Published at: {doc['published_at']}")

### Assemble the Pipeline
Now that we have the retrieval functions, we can assemble the pipeline. The pipeline will take a query as input, and retrieve the relevant documents using the BM25 and semantic search retrieval functions. The pipeline will return the retrieved documents for later use in the generation function.

In [182]:
def context_pipeline(query, top_k=5):
    bm25_results, bm25_scores = BM25_RETRIEVER.retrieve(bm25s.tokenize(query), k=top_k)
    bm25_indices = [corpus.index(doc) for doc in bm25_results[0]]

    semantic_indices = semantic_search(query, EMBEDDINGS, model, top_k=top_k)

    fused_indices = reciprocal_rank_fusion(bm25_indices, semantic_indices, top_k=top_k)
    
    fused_documents = query_news(fused_indices, NEWS_DATA)
    
    return fused_documents

# fused_documents is a list of dictionaries, each dictionary contains the title, description, and published_at of the document


## Assemble the RAG system

With the "context" for the query ready, we can now assemble the RAG system by combining the retrieval and generation components. 

We will use OpenAI's `GPT-4o-mini` model for the generation component, which is a powerful language model that can generate human-like text based on the context provided.

In [ ]:
# Initializing
import openai
import os
from openai import OpenAI
from IPython.display import Markdown, display

api_key = os.getenv("OPENAI_API_KEY") # Use correct env var name from your .env or manually set it
print("OpenAI API Key:", api_key[:10] + "..." if api_key else "Not found")  # Only show first 10 chars for security
base_url = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")  # Use correct env var name from your .env
print("OpenAI Base URL:", base_url)

# Initialize OpenAI client with API key and base URL
client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

The following function `gen_prompt()` takes a query and context as input, and generates a prompt for the language model. The prompt is formatted to include the query and the context, which will help the model generate a relevant response.

Play with the prompt template will affect the quality of the generated response. You can experiment with different prompt templates to see how they affect the output.

In [202]:
# version 1
def gen_prompt(prompt, context=None):
    # context in this case is a list of dictionaries, each dictionary contains the title, description, and published_at of the document
    # convert context to a string
    if context:
        context = "\n\n".join([f"Title: {doc['title']}\n Description: {doc['description']}\n Published at: {doc['published_at']} \n URL: {doc['url']}" for doc in context])
    return f"""
    You are a helpful assistant with access to the following context:
    ====
    Context: {context}\n
    ====\n

    User query: {prompt};
    Use the context to answer the user's question as accurately as possible.
    If the context does not provide enough information, politely inform the user that you cannot answer based on the provided context.
    """

# version 2: set up a template outside the function for easier modification
# you can add more controls too

sys_prompt = """
You are a helpful assistant with access to the following context:

======
Context: {context}
======

User query: {query};
Use the context to answer the user's question as accurately as possible.
If the context does not provide enough information, politely inform the user that you cannot answer based on the provided context.
"""

def gen_prompt_v2(query, context=None, template=sys_prompt):
    #check if context is a list of dictionaries, if so, convert it to a string
   if isinstance(context, list) and all(isinstance(doc, dict) for doc in context):
       # Convert each document in the context to a formatted string
       formated_context = "\n\n".join([f"Title: {doc['title']}\n Description: {doc['description']}\n Published at: {doc['published_at']} \n URL: {doc['url']}" for doc in context])
       print("Formatted context for prompt:\n", formated_context)  # Debugging line to check the formatted context

   else:
       # If context is None or not a list of dictionaries, set it to an empty string
       formated_context = context
   final_prompt = template.format(context=formated_context, query=query)
   return final_prompt

In [203]:
query = "What are the recent news about wheat in China?"
context = context_pipeline(query)
prompt = gen_prompt_v2(query, context = context)
print(prompt)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]


You are a helpful assistant with access to the following context:

Context: [guid                             f2066c4174cd53513551420d0a4afbab
title           China's Gold Consumption Rises on Safe-Haven D...
description     Chinese buyers, spooked by a protracted proper...
venue                                                         WSJ
url             https://www.wsj.com/articles/chinas-gold-consu...
published_at                               2024-04-26 09:40:00+00
updated_at                          2024-04-27 05:32:46.196296+00
Name: 488, dtype: object, guid                             a1e9503e615f7fa4d4ea8fab6cf6c76a
title                      Blinken’s Visit to China: What to Know
description     Secretary of State Antony J. Blinken is in Chi...
venue                                                         NYT
url             https://www.nytimes.com/2024/04/25/world/asia/...
published_at                               2024-04-25 04:46:13+00
updated_at                          20

/Users/maggiezhao/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py:1541: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [192]:
def generate_llm_with_rag(prompt, api_key=api_key, base_url=base_url, use_RAG=True, top_k=3, print_context=False):

    client = OpenAI(api_key=api_key, base_url=base_url)
    if use_RAG:
        context = context_pipeline(query, top_k=top_k)  # Get the context using the index_pipeline function
        prompt = gen_prompt_v2(prompt, context=context)  # Use the sys_prompt function to format the prompt with RAG data
        response = client.chat.completions.create(
        model="gpt-4o-mini",  # or "gpt-4o-mini-preview" for the preview version
        temperature=0.7,
        max_tokens=500,
        top_p=1.0,
        messages=[{"role": "user", "content": prompt}]
    )
        if print_context:
            print("Context used for RAG:", context)  # Print the context if requested
    
        return response.choices[0].message.content
    else:
        # If not using RAG, just return the prompt
        response = client.chat.completions.create(
        model="gpt-4o-mini",  # or "gpt-4o-mini-preview" for the preview version
        temperature=0.7,
        max_tokens=500,
        top_p=1.0,
        messages=[{"role": "user", "content": prompt}]
    )
        return response.choices[0].message.content

In [204]:
# Generate response using the LLM with RAG
 
user_input = input("You: ")
print("You:", user_input)
# Initialize messages with the user's input
response_generate_llm_with_rag = generate_llm_with_rag(user_input, api_key=api_key, base_url=base_url, use_RAG=True,print_context=True)
print("RAG Assistant:")
display(Markdown(response_generate_llm_with_rag))


You: What are the recent news about China?


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Context used for RAG: [guid                             f2066c4174cd53513551420d0a4afbab
title           China's Gold Consumption Rises on Safe-Haven D...
description     Chinese buyers, spooked by a protracted proper...
venue                                                         WSJ
url             https://www.wsj.com/articles/chinas-gold-consu...
published_at                               2024-04-26 09:40:00+00
updated_at                          2024-04-27 05:32:46.196296+00
Name: 488, dtype: object, guid                             a1e9503e615f7fa4d4ea8fab6cf6c76a
title                      Blinken’s Visit to China: What to Know
description     Secretary of State Antony J. Blinken is in Chi...
venue                                                         NYT
url             https://www.nytimes.com/2024/04/25/world/asia/...
published_at                               2024-04-25 04:46:13+00
updated_at                          2024-04-26 20:03:01.551287+00
Name: 97, dtype: object, gu

Recent news about China includes:

1. **China's Gold Consumption**: There has been a notable rise in gold consumption among Chinese buyers, driven by concerns over economic stability and a protracted property downturn. This trend reflects a shift towards gold as a safe-haven asset in uncertain times. More details can be found in the article titled "China's Gold Consumption Rises on Safe-Haven Demand" published by the Wall Street Journal on April 26, 2024. [Read more here](https://www.wsj.com/articles/chinas-gold-consu).

2. **Blinken's Visit to China**: U.S. Secretary of State Antony J. Blinken recently visited China, where he addressed various diplomatic matters. The visit is significant in the context of U.S.-China relations and ongoing global issues. The New York Times reported on this visit on April 25, 2024. [Read more here](https://www.nytimes.com/2024/04/25/world/asia/...).

3. **Warnings Against Supporting Russia**: During his visit, Blinken is expected to warn China against providing support to Russia amid the ongoing conflict in Ukraine. This is part of broader discussions on international relations and security concerns. This topic was covered by RT on April 20, 2024. [Read more here](https://www.rt.com/news/596329-blinken-to-thre...).

These articles highlight key developments in China's economic behavior and its diplomatic engagements.

In [ ]:
# Generate response using the LLM without RAG
user_input = input("You: ")
print("You:", user_input)

response_generate_llm_with_rag_no_rag = generate_llm_with_rag(user_input, api_key=api_key, base_url=base_url, use_RAG=False)
print("Assistant (without RAG):")
display(Markdown(response_generate_llm_with_rag_no_rag))

=========END=========